In [ ]:
!pip install chromadb --quiet

In [ ]:
import itertools
import boto3
import chromadb
import re
from chromadb.utils.embedding_functions import AmazonBedrockEmbeddingFunction

# Helper functions
def get_collection(path, collection_name):
    session = boto3.Session()
    embedding_function = AmazonBedrockEmbeddingFunction(
        session=session,
        model_name="amazon.titan-embed-text-v2:0")
    
    client = chromadb.PersistentClient(path=path)
    collection = client.get_collection(collection_name, embedding_function=embedding_function)
    
    return collection

def get_vector_search_results(collection, question):
    # Perform the search query
    results = collection.query(
        query_texts=[question],
        n_results=4
    )
    
    # Extract documents and distances
    documents = results.get('documents', [])[0]  # Extract the list of documents
    distances = results.get('distances', [])[0]  # Extract the list of distances
    
    return {
        'documents': documents,
        'distances': distances #Euclidean Distance
    }

In [ ]:
#startup script to populate vector db

def get_text_embeddings_collection(collection_name):
    session = boto3.Session()
    embedding_function = AmazonBedrockEmbeddingFunction(session=session, model_name="amazon.titan-embed-text-v2:0")
    
    client = chromadb.PersistentClient(path="/home/sagemaker-user/data/chroma") #replace /home/sagemaker-user with absolute path of your own environment
    index = client.get_or_create_collection(collection_name, embedding_function=embedding_function)
    
    return index


def initialize_collection(collection_name, source_json_file):
    
    collection = get_text_embeddings_collection(collection_name)
    
    if collection.count() == 0:
        
        row_count = 0
        
        with open(source_json_file) as json_file:
            source_json = json.load(json_file)
            
            for item in source_json:
                row_count = row_count + 1
                collection.add(
                    ids=[str(item['id'])],
                    documents=[item['document']],
                    metadatas=[item['metadata']],
                    embeddings=[item['embedding']]
                )
    
    print(f"Initialized collection {collection_name}")
    
    return collection



initialize_collection('services_collection', 'services_with_embeddings.json')

initialize_collection('bedrock_faqs_collection', 'bedrock_faqs_with_embeddings.json')

In [ ]:
import re
import itertools
import boto3

def get_rag_response(question):
    session = boto3.Session()
    bedrock = session.client(service_name='bedrock-runtime')
    
    collection = get_collection("/home/sagemaker-user/data/chroma", "bedrock_faqs_collection") #replace /home/sagemaker-user with absolute path of your own environment
    
    search_results = get_vector_search_results(collection, question)
    
    # Extract documents and distances
    documents = search_results.get('documents', [])
    distances = search_results.get('distances', [])
    
    # Format the documents
    rag_content = "\nNew document:\n".join(documents)
    
    # Clean up rag_content by removing extra newlines
    rag_content = re.sub(r'\n\s*\n', '\n\n', rag_content).strip()
    
    # Format similarity scores
    formatted_scores = "\n".join([f"Document {i+1}: Euclidean Distance = {distance:.4f}" for i, distance in enumerate(distances)])
    
    message = {
        "role": "user",
        "content": [
            { "text": rag_content },
            { "text": "Based on the content above, please answer the following question:" },
            { "text": question }
        ]
    }
    
    print("Formatted Message:")
    print("=" * 40)
    
    # Print RAG Content
    print("RAG Content:")
    print("=" * 40)
    print(rag_content)
    print()
    
    # Print Similarity Scores
    print("Similarity Scores:")
    print("=" * 40)
    if distances:
        print(formatted_scores)
    else:
        print("No similarity scores available.")
    print()
    
    # Print Instruction
    print("Instruction:")
    print("=" * 40)
    print("Based on the content above, please answer the following question:")
    print()
    
    # Print User Question
    print("User Question:")
    print("=" * 40)
    print(question.strip())  # Strip to remove leading/trailing whitespace
    
    response = bedrock.converse(
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",
        messages=[message],
        inferenceConfig={
            "maxTokens": 2000,
            "temperature": 0,
            "topP": 0.9,
            "stopSequences": []
        },
    )
    
    return response['output']['message']['content'][0]['text'], rag_content, formatted_scores

# Example usage
final_answer, rag_content, scores = get_rag_response(question="What are Guardrails")

In [ ]:
print("Final Answer:")
print("=" * 40)
print(final_answer.strip())